## [安装python-can](https://python-can.readthedocs.io/en/master/installation.html)

## 视频教程：[手把手教你用Python控制小米微电机](https://www.bilibili.com/video/BV1Qu411P73S)

In [1]:
import struct
import can
import time


class CANMotorController:
    PARAMETERS = {
        "run_mode": {"index": 0x7005, "format": "u8"},
        "iq_ref": {"index": 0x7006, "format": "f"},
        "spd_ref": {"index": 0x700A, "format": "f"},
        "limit_torque": {"index": 0x700B, "format": "f"},
        "cur_kp": {"index": 0x7010, "format": "f"},
        "cur_ki": {"index": 0x7011, "format": "f"},
        "cur_filt_gain": {"index": 0x7014, "format": "f"},
        "loc_ref": {"index": 0x7016, "format": "f"},
        "limit_spd": {"index": 0x7017, "format": "f"},
        "limit_cur": {"index": 0x7018, "format": "f"},
    }
    TWO_BYTES_BITS = 16

    def __init__(self, bus, motor_id=127, main_can_id=254):
        self.bus = bus
        self.MOTOR_ID = motor_id
        self.MAIN_CAN_ID = main_can_id
        self.P_MIN = -12.5
        self.P_MAX = 12.5
        self.V_MIN = -30.0
        self.V_MAX = 30.0
        self.KP_MIN, self.KP_MAX = 0.0, 500.0  # 0.0 ~ 500.0
        self.KD_MIN, self.KD_MAX = 0.0, 5.0  # 0.0 ~ 5.0

    # 通信类型
    class CmdModes:
        GET_DEVICE_ID = 0
        MOTOR_CONTROL = 1
        MOTOR_FEEDBACK = 2
        MOTOR_ENABLE = 3
        MOTOR_STOP = 4
        SET_MECHANICAL_ZERO = 6
        SET_MOTOR_CAN_ID = 7
        SINGLE_PARAM_READ = 17
        SINGLE_PARAM_WRITE = 18
        FAULT_FEEDBACK = 21

    def _float_to_uint(self, x, x_min, x_max, bits):
        span = x_max - x_min
        offset = x_min
        x = max(min(x, x_max), x_min)  # Clamp x to the range [x_min, x_max]
        return int(((x - offset) * ((1 << bits) - 1)) / span)

    def _uint_to_float(self, x, x_min, x_max, bits):
        span = (1 << bits) - 1
        offset = x_max - x_min
        x = max(min(x, span), 0)  # Clamp x to the range [0, span]
        return offset * x / span + x_min

    # 定义线性映射函数
    def _linear_mapping(
        self, value, value_min, value_max, target_min=0, target_max=65535
    ):
        return int(
            (value - value_min) / (value_max - value_min) * (target_max - target_min)
            + target_min
        )

    def format_data(self, data=[], format="f f", type="decode"):
        format_list = format.split()
        rdata = []
        if type == "decode":
            p = 0
            for f in format_list:
                s_f = []
                if f == "f":
                    s_f = [4, "f"]
                elif f == "u16":
                    s_f = [2, "H"]
                elif f == "s16":
                    s_f = [2, "h"]
                elif f == "u32":
                    s_f = [4, "I"]
                elif f == "s32":
                    s_f = [4, "i"]
                elif f == "u8":
                    s_f = [1, "B"]
                elif f == "s8":
                    s_f = [1, "b"]
                ba = bytearray()
                if len(s_f) == 2:
                    for i in range(s_f[0]):
                        ba.append(data[p])
                        p = p + 1
                    rdata.append(struct.unpack(s_f[1], ba)[0])
                else:
                    print("unknown format in format_data(): " + f)
                    return []
            return rdata
        elif type == "encode" and len(format_list) == len(data):
            for i in range(len(format_list)):
                f = format_list[i]
                s_f = []
                if f == "f":
                    s_f = [4, "f"]
                elif f == "u16":
                    s_f = [2, "H"]
                elif f == "s16":
                    s_f = [2, "h"]
                elif f == "u32":
                    s_f = [4, "I"]
                elif f == "s32":
                    s_f = [4, "i"]
                elif f == "u8":
                    s_f = [1, "B"]
                elif f == "s8":
                    s_f = [1, "b"]
                if f != "f":
                    data[i] = int(data[i])
                if len(s_f) == 2:
                    bs = struct.pack(s_f[1], data[i])
                    for j in range(s_f[0]):
                        rdata.append(bs[j])
                else:
                    print("unkown format in format_data(): " + f)
                    return []
            if len(rdata) < 4:
                for i in range(4 - len(rdata)):
                    rdata.append(0x00)
            return rdata

    # 定义打包data1函数
    def pack_to_8bytes(self, target_angle, target_velocity, Kp, Kd):
        # 对输入变量进行线性映射
        target_angle_mapped = self._linear_mapping(target_angle, self.P_MIN, self.P_MAX)
        target_velocity_mapped = self._linear_mapping(
            target_velocity, self.V_MIN, self.V_MAX
        )
        Kp_mapped = self._linear_mapping(Kp, self.KP_MIN, self.KP_MAX)
        Kd_mapped = self._linear_mapping(Kd, self.KD_MIN, self.KD_MAX)

        # 使用Python的struct库进行打包
        # 使用H表示无符号短整数（2字节），共需要8字节
        data1_bytes = struct.pack(
            "HHHH", target_angle_mapped, target_velocity_mapped, Kp_mapped, Kd_mapped
        )
        data1 = [b for b in data1_bytes]
        return data1

    def send_receive_can_message(self, cmd_mode, data2, data1, timeout=200):
        """
        Send a CAN message and receive a response.
        
        Parameters:
        - cmd_mode: Command mode (bit28~bit24)
        - data2: Data area 2 (bit23~bit8)
        - data1: Data bytes to send
        - timeout: Timeout for sending the message (default is 200ms)
        
        Returns:
        A tuple containing the received message data and the received message arbitration ID, if any.
        """
        # Calculate the arbitration ID
        arbitration_id = (cmd_mode << 24) | (data2 << 8) | self.MOTOR_ID
        message = can.Message(
            arbitration_id=arbitration_id, data=data1, is_extended_id=True
        )

        # Send the CAN message
        try:
            self.bus.send(message)
        except:
            print("Failed to send the message.")
            return None, None

        # Output details of the sent message
        print(f"Sent message with ID {hex(arbitration_id)}, data: {data1}")

        # Receive a CAN message with a 1-second timeout
        received_msg = self.bus.recv(timeout=1)  # 1-second timeout for receiving
        if received_msg:
            return received_msg.data, received_msg.arbitration_id
        else:
            return None, None

    def parse_received_msg(self, data, arbitration_id):
        if data is not None:
            print(f"Received message with ID {hex(arbitration_id)}")
            pos = self._uint_to_float(
                (data[0] << 8) + data[1], self.P_MIN, self.P_MAX, self.TWO_BYTES_BITS
            )
            vel = self._uint_to_float(
                (data[2] << 8) + data[3], self.V_MIN, self.V_MAX, self.TWO_BYTES_BITS
            )
            print(f"pos:{pos:.2f} rad, vel:{vel:.2f} rad/s")
            return pos, vel
        else:
            print("No message received within the timeout period.")
            return None, None

    def clear_can_rx(self, timeout=10):
        """
        Clear the receiver buffer by reading all existing messages.
        
        Parameters:
        - timeout: Time in ms to wait for the clearing operation.
        """
        timeout_seconds = timeout / 1000.0  # Convert to seconds
        while True:
            received_msg = self.bus.recv(timeout=timeout_seconds)
            if received_msg is None:
                break
            print(f"Cleared message with ID {hex(received_msg.arbitration_id)}")

    def _write_single_param(self, index, value, format="u32"):
        encoded_data = self.format_data(data=[value], format=format, type="encode")
        data1 = [b for b in struct.pack("<I", index)] + encoded_data

        self.clear_can_rx()  # 空CAN接收缓存，避免读到老数据

        received_msg_data, received_msg_arbitration_id = self.send_receive_can_message(
            cmd_mode=self.CmdModes.SINGLE_PARAM_WRITE,
            data2=self.MAIN_CAN_ID,
            data1=data1,
        )
        self.parse_received_msg(received_msg_data, received_msg_arbitration_id)
        return received_msg_data, received_msg_arbitration_id

    def write_single_param(self, param_name, value):
        param_info = self.PARAMETERS.get(param_name)
        if param_info is None:
            print(f"Unknown parameter name: {param_name}")
            return

        index = param_info["index"]
        format = param_info["format"]

        self._write_single_param(index=index, value=value, format=format)

    def set_0_pos(self):
        self.clear_can_rx()  # 清空CAN接收缓存，避免读到老数据

        received_msg_data, received_msg_arbitration_id = self.send_receive_can_message(
            cmd_mode=self.CmdModes.SET_MECHANICAL_ZERO,
            data2=self.MAIN_CAN_ID,
            data1=[1],
        )

        self.parse_received_msg(received_msg_data, received_msg_arbitration_id)

    def enable(self):
        self.clear_can_rx(0)  # 清空CAN接收缓存，避免读到老数据

        received_msg_data, received_msg_arbitration_id = self.send_receive_can_message(
            cmd_mode=self.CmdModes.MOTOR_ENABLE, data2=self.MAIN_CAN_ID, data1=[]
        )
        self.parse_received_msg(received_msg_data, received_msg_arbitration_id)

    def disable(self):
        self.clear_can_rx(0)  # 空CAN接收缓存，避免读到老数据

        received_msg_data, received_msg_arbitration_id = self.send_receive_can_message(
            cmd_mode=self.CmdModes.MOTOR_STOP,
            data2=self.MAIN_CAN_ID,
            data1=[0, 0, 0, 0, 0, 0, 0, 0],
        )
        self.parse_received_msg(received_msg_data, received_msg_arbitration_id)

    def set_motor_position_control(self, limit_spd, loc_ref):
        # 设置电机最大速度
        self.write_single_param(param_name="limit_spd", value=limit_spd)
        # 设置电机目标位置
        self.write_single_param(param_name="loc_ref", value=loc_ref)

In [2]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(bustype="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
motor = CANMotorController(bus, motor_id=127, main_can_id=254)

## Jog
单个参数写入（通信类型 18）

In [3]:
jog_vel = 5  # rad/s
uint_value = motor._float_to_uint(jog_vel, motor.V_MIN, motor.V_MAX, 16)
jog_vel_bytes = motor.format_data(data=[uint_value], format="u16", type="encode")[:2][::-1]

data1 = [0x05, 0x70, 0x00, 0x00, 0x07, 0x01] + jog_vel_bytes
motor.clear_can_rx()
received_msg_data, received_msg_arbitration_id = motor.send_receive_can_message(
    cmd_mode=motor.CmdModes.SINGLE_PARAM_WRITE, data2=motor.MAIN_CAN_ID, data1=data1
)
motor.parse_received_msg(received_msg_data, received_msg_arbitration_id)

Sent message with ID 0x1200fe7f, data: [5, 112, 0, 0, 7, 1, 149, 84]
Received message with ID 0x2807ffe
pos:5.34 rad, vel:-0.06 rad/s


(5.344281681544214, -0.06271457999542207)

## 停止

In [4]:
motor.disable()

Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:9.87 rad, vel:4.97 rad/s


In [5]:
motor.set_0_pos()

Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:-0.10 rad/s


## 读取速度和位置
- 发送参数：单个参数写⼊（通信类型18），写入地址0x7018（电流限制）值为 27A
- 读取返回值：电机反馈数据（通信类型2），Byte0,1: 当前⻆度，Byte2,3:当前⻆速度

In [6]:
motor.write_single_param(param_name="limit_cur", value=27)

Sent message with ID 0x1200fe7f, data: [24, 112, 0, 0, 0, 0, 216, 65]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:-0.08 rad/s


# 位置模式
## 发送电机模式参数写入命令（通信类型 18）
设置 `runmode` 参数为 1
- index(Byte0~1): `run_mode`，0x7005
- value(Byte4~7): 1(位置模式)

In [7]:
motor.write_single_param("run_mode", value=1)

Sent message with ID 0x1200fe7f, data: [5, 112, 0, 0, 1, 0, 0, 0]
Received message with ID 0x2007ffe
pos:0.00 rad, vel:0.01 rad/s


In [8]:
motor.enable()

Sent message with ID 0x300fe7f, data: []
Received message with ID 0x2807ffe
pos:0.00 rad, vel:0.02 rad/s


## 最大速度：发送电机模式参数写入命令（通信类型 18）
设置 `limit_spd` 参数为预设最大速度指令
- index(Byte0~1): `limit_spd`, 0x7017
- value(Byte4~7): `float` [0,30]rad/s

In [9]:
motor.write_single_param("limit_spd", value=10)

Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 32, 65]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.01 rad/s


## 目标位置：发送电机模式参数写入命令（通信类型 18）
设置 `loc_ref` 参数为预设位置指令
- index(Byte0~1): `loc_ref`, 0x7016
- value(Byte4~7): `float` rad

In [10]:
motor.write_single_param("loc_ref", value=3.14)

Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 195, 245, 72, 64]
Received message with ID 0x2807ffe
pos:0.00 rad, vel:0.02 rad/s


In [18]:
def set_motor_position_control(limit_spd, loc_ref):
    # 设置电机最大速度
    write_single_param(index=0x7017, value=limit_spd, format="f")
    # 设置电机目标位置
    write_single_param(index=0x7016, value=loc_ref, format="f")

In [11]:
import time
# 定义音符到电机参数的映射，为后面的音符分配更高的转速值
note_to_speed = {"C": 6.6, "D": 6.8, "E": 7.3, "F": 8, "G": 9, "A": 10, "B": 13}

# 定义小星星的音符和时长
# 完整定义小星星的音符和时长
notes = ["C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C",
         "G", "G", "F", "F", "E", "E", "D",
         "G", "G", "F", "F", "E", "E", "D",
         "C", "C", "G", "G", "A", "A", "G",
         "F", "F", "E", "E", "D", "D", "C"]

durations = [1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2,
             1, 1, 1, 1, 1, 1, 2]  # 以四分音符为单位

x_speed = 1.5 # 速度倍数
x_duration = 0.4 # 间隔倍数，如果小于1，就是减小时间间隔


# 初始化电机的当前位置和方向标志
current_loc_ref = 0
direction_flag = 1  # 1表示正方向，-1表示反方向

motor.disable() #
motor.set_motor_position_control(limit_spd=10, loc_ref=0)
motor.set_0_pos()  # 当前0位
motor.enable()

# 播放小星星
for note, duration in zip(notes, durations):
    duration = duration * x_duration
    
    # 计算电机的转速和目标位置
    limit_spd = note_to_speed[note] * x_speed
    loc_ref = current_loc_ref + (direction_flag * limit_spd * duration * 1.5)

    # 使用电机参数设置函数
    motor.set_motor_position_control(limit_spd=limit_spd, loc_ref=loc_ref)

    # 更新电机的当前位置
    current_loc_ref = loc_ref

    # 反转方向标志
#     direction_flag *= -1

    # 休眠一段时间以模拟音符的时长（这里假设一个四分音符的时长是1秒）
    time.sleep(duration * x_speed)
disable()

Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:0.03 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 32, 65]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:-0.12 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:3.14 rad, vel:-0.01 rad/s
Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:0.22 rad/s
Sent message with ID 0x300fe7f, data: []
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:0.05 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:-0.01 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 123, 20, 190, 64]
Received message with ID 0x2807ffe
pos:-0.00 rad, vel:0.01 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:5.85 

Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:6.14 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 10, 215, 103, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.12 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 102, 102, 30, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.96 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 174, 199, 109, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:3.23 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 88, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:6.30 rad/s
Sent message with ID 0x1200fe7f, data: [22, 112, 0, 0, 72, 225, 117, 67]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:4.29 rad/s
Sent message with ID 0x1200fe7f, data: [23, 112, 0, 0, 0, 0, 88, 65]
Received message with ID 0x2807ffe
pos:12.50 rad, vel:8.07 rad/s
Sent message with ID 0x1200fe7f, data: [2

NameError: name 'disable' is not defined

In [12]:
motor.disable()
motor.set_0_pos()

Sent message with ID 0x400fe7f, data: [0, 0, 0, 0, 0, 0, 0, 0]
Received message with ID 0x2007ffe
pos:12.50 rad, vel:0.03 rad/s
Sent message with ID 0x600fe7f, data: [1]
Received message with ID 0x2007ffe
pos:-0.00 rad, vel:0.05 rad/s
